# Getting CDI and reduced RF metrics

Generated postcodes in google colab 
Then I extracted the postcode with excel formula - https://www.extendoffice.com/documents/excel/1660-excel-extract-postcode.html
It extracted xxxx, Australia

So I used Left value function to extract first 4 letters Source: https://www.exceldemy.com/extract-specific-numbers-from-a-cell-in-excel/ 

The CDI files came in seperate files for each month. So I wanted to combine them

Monthly CDI data by parish can be downloaded from here to complete the following section: https://edis.spaceport.intersect.org.au/%2FMonthly%20Data%2FEDISdata_byParish

### To do
a) write a program that reads one file and writes a new with your added column (for example read a line, add your new column stringwise and write) 

b) take the part that reads and wrap that into a function that returns the lines to write. 

c) open your file that you want to write to

d) make a loop over the files that you want to read, and then just write your new lines to the final file.

In [ ]:
def add_yr_mth_to_EDIS(name):

    import pandas as pd
    import re
    import datetime

    name = name

    # Extracting year and month from the name
    # year = re.search(r'(\d+)',name)
    # OR
    # using join() + isdigit() + filter()
    # Extract digit string
    year = ''.join(filter(lambda i: i.isdigit(), name)) #https://www.geeksforgeeks.org/python-extract-digits-from-given-string/
    # Now remove the last 4 and only keep the first 4
    year = year[:4] # Iterating through the first 4 numbers https://stackoverflow.com/questions/42788930/how-to-take-only-first-four-elements-of-every-strings-from-a-list
    print(year)

    # Doing the same for month
    mth = ''.join(filter(lambda i: i.isdigit(), name))
    # The second last digit and the fourth last digit
    mth = mth[-4:-2]
    
    # Converting month from num to string:
    #provide month number (Source:https://www.studytonight.com/python-howtos/how-to-get-month-name-from-month-number-in-python)
    datetime_object = datetime.datetime.strptime(mth, "%m")
    mth = datetime_object.strftime("%B")
    print(mth)


    # Read file:
    df_edis = pd.read_excel(name + '.xlsx')

    # Solution 1: Adding column with same value
    # yr_mth_dict = {'Year': 2016, 'Month': 'January'}
    #df_edis.assign(**yr_mth_dict,inplace = True) 

    # Solution 2:
    # adding column with constant value
    df_edis['Year'] = pd.Series([year for x in range(len(df_edis.index))])
    df_edis['Month'] = pd.Series([mth for x in range(len(df_edis.index))])
    
    df_edis.to_excel(str(name) + '_yr_mth.xlsx')


In [ ]:
##Combining postcode reports
import glob
import os
##Setting path of files and giving name
files = os.path.join("edis_*.xlsx")
##List of merged files returned
files = glob.glob(files)
print(files)

## Add year and month columns to files
for file in files:
   file_name = file.split('.') # Source: https://www.pythonpool.com/python-get-filename-without-extension/
   print(file_name)
   file_name = file_name[0]
   print(file_name)
   add_yr_mth_to_EDIS(file_name)


Moved all the old files back into a different folder
Now I only have the new files

Now I am going to combine all the EDIS files

In [ ]:
##Combining EDIS reports
import pandas as pd
import glob
import os
##Setting path of files and giving name
files = os.path.join("edis_*.xlsx")
##List of merged files returned
files = glob.glob(files)
files
## Join files with concat
df_files = pd.concat(map(pd.read_excel, files), ignore_index=True)
# print(df_files)
##Saving to csv
df_files.to_csv('Combined_EDIS_2016_01_to_2021_12.csv')


In [ ]:
import pandas as pd
# Opening database with postcodes 935 points
db_postcode = pd.read_csv('Database_All_AWRA_NDVI_MOf_and_3MPrecip_noWel_postcode_data_2022_09_21.csv')
#db_postcode.info()
# Opening EDIS with postcodes
df_files = pd.read_csv('Combined_EDIS_2016_01_to_2021_12.csv')
# Capitalise Location of db_postcode OR make only the first letter capital and rest lower case in EDIS: Source:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.capitalize.html
df_files['parishname'] = df_files['parishname'].str.title() #If we just do the right side, it does not save it so we assign the same column back to the function we do to it to save it - example: https://datatofish.com/uppercase-pandas-dataframe/
#df_files.info()

# Renaming columns so can merge:
df_files.rename(columns={'parishname':'Location','postcode':'postal_code'},inplace=True)
#df_files.info()

# Converting obj to string because when you merge on obj it can stuff up.
db_postcode['postal_code'] = db_postcode['postal_code'].astype(str)
df_files['postal_code'] = df_files['postal_code'].astype(str)

# Merging:
# https://www.statology.org/pandas-merge-multiple-columns/
result = db_postcode.merge(df_files[['Location','CDI','Year','Month','postal_code']], on = ['postal_code','Year','Month'], how = 'left')
# Dropping rows that dont meet below condition:
result2 = result[result.Location_x == result.Location_y]
result2.to_csv('Database_All_AWRA_NDVI_MOf_and_3MPrecip_noWel_postcode_MERGED_CDI_data_2022_09_22.csv')

I made a database with CDI from the LGAs instead. I got much more. Now I will run the metrics on it. 

3 Oct 2022

In [ ]:
# 5th October - redoing so that I can get lga and countyname for all locations. 
import pandas as pd
# Opening database with postcodes 935 points
db_postcode = pd.read_csv('Database_All_AWRA_NDVI_MOf_and_3MPrecip_noWel_postcode_data_2022_09_21.csv')
#db_postcode.info()
# Opening EDIS with postcodes
df_files = pd.read_csv('Combined_EDIS_2016_01_to_2021_12.csv')
# Capitalise Location of db_postcode OR make only the first letter capital and rest lower case in EDIS: Source:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.capitalize.html
df_files['parishname'] = df_files['parishname'].str.title() #If we just do the right side, it does not save it so we assign the same column back to the function we do to it to save it - example: https://datatofish.com/uppercase-pandas-dataframe/
#df_files.info()

# Renaming columns so can merge:
df_files.rename(columns={'parishname':'Location','postcode':'postal_code'},inplace=True)
#df_files.info()

# Converting obj to string because when you merge on obj it can stuff up.
db_postcode['postal_code'] = db_postcode['postal_code'].astype(str)
df_files['postal_code'] = df_files['postal_code'].astype(str)

# Merging:
# https://www.statology.org/pandas-merge-multiple-columns/
result = db_postcode.merge(df_files[['countyname','lganame','Location','CDI','Year','Month','postal_code']], on = ['postal_code','Year','Month'], how = 'left')
# Dropping rows that dont meet below condition:
result2 = result[result.Location_x == result.Location_y]
result2.to_csv('Database_All_AWRA_NDVI_MOf_and_3MPrecip_noWel_postcode_MERGED_CDI_data_2022_10_05.csv')

Manually sorted into Database_All_AWRA_NDVI_MOf_and_3MPrecip_noWel_postcode_MERGED_CDI_from_LGA_names_manual_sorting_data_2022_09_28

Then I also redid the parishname code and cleaned it checking with the arcgis map each coordinate and then I also combined the LGA one and the other one to get 702 lines. I removed any duplicates. So it is a slightly different dataset. It is only the 2018,2019,2020 and 2021 datapoints from the original dataset.

In [ ]:
def get_metrics_for_CDI(database_file_name, CDI_number):

    ## IMPORTING:
    # Importing numpy
    import numpy as np
    import pandas as pd
    # For plotting feature importance: 
    import matplotlib.pyplot as plt
    import time
    # For dividing training and testing  
    from sklearn.model_selection import train_test_split
    #For training the model
    from sklearn.ensemble import RandomForestClassifier
    # For metrics
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    # For feature importance plots:
    from sklearn.inspection import permutation_importance

    # Assign variables:
    database_file_name = database_file_name
    CDI_number = CDI_number #This can be 7,6,5,4,3. USed for y_pred_0_test etc

    # The name of the numbers included to define drought 0=drought(1), 0,1 = drought(1); 0,1,2=drought(1) and so on
    CDI_thresholds = [0,1,2,'01234','0123','012','01','0'] #0 = -7 or CDI_number of 7
    threshold_number = CDI_thresholds[CDI_number]

    # Opening database with CDI values and CDI0,1234 values
    db_CDI = pd.read_csv(database_file_name + '.csv') #Database_All_AWRA_NDVI_MOf_and_3MPrecip_noWel_postcode_MERGED_CDI_data_manual_check_2022_10_05.csv
    db_CDI.drop(columns = ['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0','Unnamed: 0.1'],inplace=True)  
    #This is specific to CDI file so make sure you have the correct one. 

    # Dividing data into predictors (X) and no drought/drought (y)
    y = db_CDI.iloc[:,7].values #All rows and only the 7th column (Drought no drought)
   
    X = db_CDI 
    X.drop(['Drought / No Drought'],axis = 1,inplace=True)
    #Convert to array
    X.to_numpy()

    # ----------------------------TRAINING AND RESULTS WITH DIFF RANDOM STATES---------------------------------------------------------------------------------------------------------------------------
    # Creating a dictionary called list_metrics
    acc_lm_CDI = []
    tp_lm_CDI = []
    tn_lm_CDI = []
    fp_lm_CDI = []
    fn_lm_CDI = []

    tpr_lm = []
    tnr_lm = []
    ppv_lm = []
    npv_lm = []
    fpr_lm =[]
    fnr_lm  = []
    fdr_lm = []
    acc_conf_lm = []

    for i in range(51): 
        # ----------------------------SPLITTING TRAINING AND TESTING DATASET--------------------------------------------
        #Dividing data into training and testing: 
        # from sklearn.model_selection import train_test_split
        # I put it above
    
        X_train_all, X_test_all, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= i)
        
        # Then we create the real X_train and X_test by removing those columns. This allows us to more easily see which ones were an issue. 
        X_train = X_train_all.drop(['Year_Month','Latitude','Longitude','Year','Month','Geographic Location','Location_x', 'MYD13A3_061__1_km_monthly_EVI','MYD13A3_061__1_km_monthly_NDVI','MYD13A3_061__1_km_monthly_VI_Quality_VI_Usefulness_Description','IPO_TPI'],axis = 1)
        X_test = X_test_all.drop(['Year_Month','Latitude','Longitude','Year','Month','Geographic Location','Location_x','MYD13A3_061__1_km_monthly_EVI','MYD13A3_061__1_km_monthly_NDVI','MYD13A3_061__1_km_monthly_VI_Quality_VI_Usefulness_Description','IPO_TPI'],axis = 1) 
        
        # To remove the last 10 rows that include CDI values:
        # df.iloc[row_start:row_end , col_start:col_end] 
        # Source: https://thispointer.com/pandas-delete-last-column-of-dataframe-in-python/#:~:text=Use%20drop()%20to%20remove,last%20column%20of%20pandas%20dataframe.
        X_train = X_train.iloc[:,0:10]
        X_test =  X_test.iloc[:,0:10]
        # Thus I took all rows and columns from 0 to 10 which means it deleted the last CDI rows. But I had to manually count them. 
        
        # --------------------------------TRAINING THE MODEL---------------------------------------------        
        # Training the model
        # from sklearn.ensemble import RandomForestClassifier # See above
        classifier = RandomForestClassifier(n_estimators=20, random_state=i)
        classifier.fit(X_train, y_train)
        y_pred_classifier = classifier.predict(X_test)
        
        # ----------------------------- GETTING PREDICTIONS------------------------
        #predictions for X_test
        predictions = classifier.predict_proba(X_test)
        # predictions # Col 1 is 0 and Col 2 is 1. We can
        prob_drought = predictions[:,1]
        # prob_drought
        X_test_all['ypred'] = prob_drought
        X_test_all['y_obs'] = y_test
        #X_test_all.to_csv('X_test_all_'+ str(i) +'_5Oct.csv')

        # predictions for X_train to make sure it is correct. This helps to check if it is classifying D/no D correctly
        prediction_train = classifier.predict_proba(X_train)
        # prediction_train # Col 1 is 0 and Col 2 is 1. We can
        prob_drought_train = prediction_train[:,1]
        
        X_train_all['ypred'] = prob_drought_train
        X_train_all['y_obs'] = y_train
        #X_train_all.to_csv('X_train_all_'+ str(i) +'_5Oct.csv')
        
        # ----------------------------GETTING METRICS-------------------------------------------------------
        # confusion matrix + other matrix
        # from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
        # See above

        # For confusion matrix later: https://datascience.stackexchange.com/questions/87654/is-there-a-flexible-way-to-get-the-original-data-indices-from-each-cell-of-a-con
            
        # False Positives:
        # I got inspiration from this: https://datascience.stackexchange.com/questions/87654/is-there-a-flexible-way-to-get-the-original-data-indices-from-each-cell-of-a-con
        #mistakes = np.invert(y_pred_classifier == y_test)
        #count_fp = np.count_nonzero(mistakes)
        #fp_list_metrics = np.append(fp_list_metrics,count_fp)
        # I later found that it was just calculating mistakes and not false positive.

        # FOR CDI
        y_true_test = y_test #All rows and only the Drought/No drought col
        y_pred_CDI_test = X_test_all.iloc[:,-CDI_number].values #0 = -5,0 and 1 = -4; 0 to 2 = -3; 0 to 3 = -2; 0 to 4= -1

        #y_pred_0_test = X_test_all.iloc[:,-5].values
        #y_pred_1_test = X_test_all.iloc[:,-4].values
        #y_pred_2_test = X_test_all.iloc[:,-3].values
        #y_pred_3_test = X_test_all.iloc[:,-2].values
        #y_pred_4_test = X_test_all.iloc[:,-1].values

        y_true_train = y_train #All rows and only the Drought/No drought col
        y_pred_CDI_train = X_train_all.iloc[:,-CDI_number].values
        #y_pred_0_train = X_train_all.iloc[:,-5].values
        #y_pred_1_train = X_train_all.iloc[:,-4].values
        #y_pred_2_train = X_train_all.iloc[:,-3].values
        #y_pred_3_train = X_train_all.iloc[:,-2].values
        #y_pred_4_train = X_train_all.iloc[:,-1].values
        
        # Confusion matrix for CDI - true neg, false pos, false neg, true pos:
        tn_CDI, fp_CDI, fn_CDI, tp_CDI = confusion_matrix(y_true_test, y_pred_CDI_test, labels=[0, 1]).ravel()
        
        # I wanted to automatically calculate the rates as well. Source: https://stackoverflow.com/questions/50666091/true-positive-rate-and-false-positive-rate-tpr-fpr-for-multi-class-data-in-py
        fp_CDI = fp_CDI.astype(float)
        fn_CDI = fn_CDI.astype(float)
        tp_CDI = tp_CDI.astype(float)
        tn_CDI = tn_CDI.astype(float)

        # Sensitivity, hit rate, recall, or true positive rate
        TPR = tp_CDI/(tp_CDI+fn_CDI)
        # Specificity or true negative rate
        TNR = tn_CDI/(tn_CDI+fp_CDI) 
        # Precision or positive predictive value
        PPV = tp_CDI/(tp_CDI+fp_CDI)
        # Negative predictive value
        NPV = tn_CDI/(tn_CDI+fn_CDI)
        # Fall out or false positive rate
        FPR = fp_CDI/(fp_CDI+tn_CDI)
        # False negative rate
        FNR = fn_CDI/(tp_CDI+fn_CDI)
        # False discovery rate
        FDR = fp_CDI/(tp_CDI+fp_CDI)
        # Overall accuracy
        ACC_conf = (tp_CDI+tn_CDI)/(tp_CDI+fp_CDI+fn_CDI+tn_CDI)

        tn_lm_CDI = np.append(tn_lm_CDI,tn_CDI)
        fp_lm_CDI = np.append(fp_lm_CDI,fp_CDI)
        fn_lm_CDI = np.append(fn_lm_CDI,fn_CDI)
        tp_lm_CDI = np.append(tp_lm_CDI,tp_CDI)
        tpr_lm = np.append(tpr_lm, TPR)
        tnr_lm = np.append(tnr_lm, TNR)
        ppv_lm = np.append(ppv_lm, PPV)
        npv_lm = np.append(npv_lm, NPV)
        fpr_lm = np.append(fpr_lm, FPR)
        fnr_lm  = np.append(fnr_lm, FNR)
        fdr_lm = np.append(fdr_lm, FDR)
        acc_conf_lm = np.append(acc_conf_lm, ACC_conf)

        # Accuracy for CDI:
        acc_CDI = accuracy_score(y_true_test, y_pred_CDI_test)
        acc_lm_CDI= np.append(acc_lm_CDI,acc_CDI)

        ## CDI Classification report:
        # Source: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format
        # Function
        def CDI_classification_report_csv(report,rd_state):
            report_data = []
            lines = report.split('\n')
            for line in lines[2:-5]:
                row = {}
                
                row_data = ' '.join(line.split())   
                row_data = row_data.split(' ')
                
                row['class'] = row_data[0]
                row['precision'] = float(row_data[1])
                row['recall'] = float(row_data[2])
                row['f1_score'] = float(row_data[3])
                row['support'] = float(row_data[4])
                report_data.append(row)
                # return report_data #https://stackoverflow.com/questions/3052793/python-output-from-functions

            dataframe = pd.DataFrame.from_dict(report_data)
            dataframe.to_csv('CDI_threshold'+ str(threshold_number) +'_classification_report_test_10Oct2022_'+ str(rd_state) + '.csv', index = False)
        
        report = classification_report(y_true_test, y_pred_CDI_test)
        CDI_classification_report_csv(report,i) # Previously I assigned it a variable but it returns a csv anyway so I dont need to.

        
    # It is cheaper to fill out a list than fill out a dataframe: https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-then-filling-it
    # For CDI:
    df_acc_CDI = pd.DataFrame(acc_lm_CDI)
    df_acc_CDI.columns = ['Accuracy_CDI_'+ str(threshold_number)]
    df_acc_CDI.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_Acc_test_10Oct.csv')
        
    df_tn_CDI = pd.DataFrame(tn_lm_CDI)
    df_tn_CDI.columns = ['True Negative_CDI_'+ str(threshold_number)]
    df_tn_CDI.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_trueneg_test_10Oct.csv')

    df_fp_CDI = pd.DataFrame(fp_lm_CDI)
    df_fp_CDI.columns = ['False Positive_CDI_'+ str(threshold_number)]
    df_fp_CDI.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_falsepos_test_10Oct.csv')

    df_fn_CDI = pd.DataFrame(fn_lm_CDI)
    df_fn_CDI.columns = ['False Negative_CDI_'+ str(threshold_number)]
    df_fn_CDI.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_falseneg_test_10Oct.csv')

    df_tp_CDI = pd.DataFrame(tp_lm_CDI)
    df_tp_CDI.columns = ['True Positive_CDI_'+ str(threshold_number)]
    df_tp_CDI.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_truepos_test_10Oct.csv')

    tpr_lm_df = pd.DataFrame(tpr_lm)
    tpr_lm_df.columns = ['True Positive Rate / Sensitivity_CDI_'+ str(threshold_number)]
    tpr_lm_df.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_tpr_10Oct.csv')
    tnr_lm_df = pd.DataFrame(tnr_lm)
    tnr_lm_df.columns = ['True Negative Rate / Specificity_CDI_'+ str(threshold_number)]
    tnr_lm_df.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_tnr_10Oct.csv')
    ppv_lm_df = pd.DataFrame(ppv_lm)
    ppv_lm_df.columns = ['Positive predictive value / Precision_CDI_'+ str(threshold_number)]
    ppv_lm_df.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_ppv_10Oct.csv')
    npv_lm_df = pd.DataFrame(npv_lm)
    npv_lm_df.columns = ['Negative predictive value_CDI_'+ str(threshold_number)]
    npv_lm_df.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_npv_10Oct.csv')
    fpr_lm_df = pd.DataFrame(fpr_lm)
    fpr_lm_df.columns = ['False Positive rate / Fall out_CDI_'+ str(threshold_number)]
    fpr_lm_df.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_fpr_10Oct.csv')
    fnr_lm_df = pd.DataFrame(fnr_lm)
    fnr_lm_df.columns = ['False Negative rate_CDI_'+ str(threshold_number)]
    fnr_lm_df.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_fnr_10Oct.csv')
    fdr_lm_df = pd.DataFrame(fdr_lm)
    fdr_lm_df.columns = ['False discovery rate_CDI_'+ str(threshold_number)]
    fdr_lm_df.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_fdr_10Oct.csv')
    acc_conf_lm_df = pd.DataFrame(acc_conf_lm)
    acc_conf_lm_df.columns = ['Overall accuracy (from conf m)_CDI_'+ str(threshold_number)]
    acc_conf_lm_df.to_csv('CDI_'+ str(threshold_number) +'_Random_state_0_to_50_acc_conf_10Oct.csv')

    # CDI
    ##Combining classification reports
    import glob
    import os
    ##Setting path of files and giving name
    files_CDI = os.path.join('CDI_threshold'+ str(threshold_number) +'_classification_report_test_10Oct2022_*.csv')
    ##List of merged files returned
    files_CDI = glob.glob(files_CDI)
    ## Join files with concat
    df_files_CDI = pd.concat(map(pd.read_csv, files_CDI), ignore_index=True)
    # print(df_files)
    ##Saving to csv
    df_files_CDI.to_csv('Final_CDI_'+ str(threshold_number) +'_Random_state_0_to_50_Classification_Report_10Oct.csv')

   
    ## Combining Acc and Confusion Matrix: Source:https://www.statology.org/pandas-merge-multiple-dataframes/
    from functools import reduce

    #define list of DataFrames
    dfs = [df_acc_CDI, df_fp_CDI, df_fn_CDI, df_tp_CDI , df_tn_CDI, tpr_lm_df,tnr_lm_df,ppv_lm_df,npv_lm_df,fpr_lm_df,fnr_lm_df,fdr_lm_df,acc_conf_lm_df]

    #merge all DataFrames into one
    #final_df = reduce(lambda  left,right: pd.merge(left,right,how='left'), dfs)
    # Trying to merge on index singe the above merges it by attaching it below. 
    final_df = reduce(lambda  left,right: pd.merge(left,right,left_index = True,right_index=True, how='outer'), dfs)

    final_df.to_csv('Combined_Metrics_Accuracy_Conf_Matrix_Rates_CDI_threshold'+ str(threshold_number) +'_0_to_50_test_10Oct.csv')

In [ ]:
# The column number
CDI_threshold_col_numbers = [7,6,5,4,3]

database_CDI = 'Database_All_AWRA_NDVI_MOf_and_3MPrecip_noWel_postcode_MERGED_CDI_data_manual_check_2022_10_05'
for CDI_threshold_col_number in CDI_threshold_col_numbers:
    get_metrics_for_CDI(database_CDI,CDI_threshold_col_number)

Getting RF metrics for this database

In [ ]:
def get_metrics_for_TEST_data(database,name_of_indice):

    ## IMPORTING:
    # Importing numpy
    import numpy as np
    import pandas as pd
    # For plotting feature importance: 
    import matplotlib.pyplot as plt
    import time
    # For dividing training and testing  
    from sklearn.model_selection import train_test_split
    #For training the model
    from sklearn.ensemble import RandomForestClassifier
    # For metrics
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    # For feature importance plots:
    from sklearn.inspection import permutation_importance

    # Assign variables:
    database = database
    name_of_indice = name_of_indice
    
    # Dividing data into predictors (X) and no drought/drought (y)
    y = database.iloc[:,7].values #All rows and only the 9th column (Drought no drought)
    X = database 
    X.drop(['Drought / No Drought'],axis = 1,inplace=True)
    #Convert to array
    X.to_numpy()

    # ----------------------------TRAINING AND RESULTS WITH DIFF RANDOM STATES---------------------------------------------------------------------------------------------------------------------------
    # Creating a dictionary called list_metrics
    acc_list_metrics = []
    tp_list_metrics = []
    tn_list_metrics = []
    fp_list_metrics = []
    fn_list_metrics = []

    tpr_lm = []
    tnr_lm = []
    ppv_lm = []
    npv_lm = []
    fpr_lm = []
    fnr_lm  = []
    fdr_lm = []
    acc_conf_lm = []


    for i in range(51): 
        # ----------------------------SPLITTING TRAINING AND TESTING DATASET--------------------------------------------
        #Dividing data into training and testing: 
        # from sklearn.model_selection import train_test_split
        # I put it above
    
        X_train_all, X_test_all, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= i)

        # Then we create the real X_train and X_test by removing those columns. This allows us to more easily see which ones were an issue. 
        X_train = X_train_all.drop(['Year_Month','Latitude','Longitude','Year','Month','Geographic Location','Location_x','MYD13A3_061__1_km_monthly_EVI','MYD13A3_061__1_km_monthly_NDVI','MYD13A3_061__1_km_monthly_VI_Quality_VI_Usefulness_Description','IPO_TPI'],axis = 1)
        X_test = X_test_all.drop(['Year_Month','Latitude','Longitude','Year','Month','Geographic Location','Location_x','MYD13A3_061__1_km_monthly_EVI','MYD13A3_061__1_km_monthly_NDVI','MYD13A3_061__1_km_monthly_VI_Quality_VI_Usefulness_Description','IPO_TPI'],axis = 1) 
        # To remove the last 10 rows that include SPI values:
        # df.iloc[row_start:row_end , col_start:col_end] 
        # Source: https://thispointer.com/pandas-delete-last-column-of-dataframe-in-python/#:~:text=Use%20drop()%20to%20remove,last%20column%20of%20pandas%20dataframe.

        X_train = X_train.iloc[:,0:10] 
        X_test =  X_test.iloc[:,0:10] 
        # Thus I took all rows and columns from 0 to 10 which means it deleted the last SPI rows. But I had to manually count them. 

        # --------------------------------TRAINING THE MODEL---------------------------------------------        
        # Training the model
        # from sklearn.ensemble import RandomForestClassifier # See above
        classifier = RandomForestClassifier(n_estimators=20, random_state=i)
        classifier.fit(X_train, y_train)
        y_pred_classifier = classifier.predict(X_test)
        
        # ----------------------------- GETTING PREDICTIONS------------------------
        #predictions for X_test
        predictions = classifier.predict_proba(X_test)
        # predictions # Col 1 is 0 and Col 2 is 1. We can
        prob_drought = predictions[:,1]
        # prob_drought
        X_test_all['ypred'] = prob_drought
        X_test_all['y_obs'] = y_test
        #X_test_all.to_csv('X_test_all_'+ str(i) +'_22Sept.csv')

        # predictions for X_train to make sure it is correct. This helps to check if it is classifying D/no D correctly
        prediction_train = classifier.predict_proba(X_train)
        # prediction_train # Col 1 is 0 and Col 2 is 1. We can
        prob_drought_train = prediction_train[:,1]
        
        X_train_all['ypred'] = prob_drought_train
        X_train_all['y_obs'] = y_train
        #X_train_all.to_csv('X_train_all_'+ str(i) +'_22Sept.csv')
        
        # ----------------------------GETTING METRICS-------------------------------------------------------
        # confusion matrix + other matrix
        # from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
        # See above

        # For confusion matrix later: https://datascience.stackexchange.com/questions/87654/is-there-a-flexible-way-to-get-the-original-data-indices-from-each-cell-of-a-con
            
        # False Positives:
        # I got inspiration from this: https://datascience.stackexchange.com/questions/87654/is-there-a-flexible-way-to-get-the-original-data-indices-from-each-cell-of-a-con
        #mistakes = np.invert(y_pred_classifier == y_test)
        #count_fp = np.count_nonzero(mistakes)
        #fp_list_metrics = np.append(fp_list_metrics,count_fp)
        # I later found that it was just calculating mistakes and not false positive.
        

        # Confusion matrix - true neg, false pos, false neg, true pos:
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_classifier, labels=[0, 1]).ravel()

        tn_list_metrics = np.append(tn_list_metrics,tn)
        fp_list_metrics = np.append(fp_list_metrics,fp)
        fn_list_metrics = np.append(fn_list_metrics,fn)
        tp_list_metrics = np.append(tp_list_metrics,tp)

        # RATES:
        # Sensitivity, hit rate, recall, or true positive rate
        TPR = tp/(tp+fn)
        # Specificity or true negative rate
        TNR = tn/(tn+fp) 
        # Precision or positive predictive value
        PPV = tp/(tp+fp)
        # Negative predictive value
        NPV = tn/(tn+fn)
        # Fall out or false positive rate
        FPR = fp/(fp+tn)
        # False negative rate
        FNR = fn/(tp+fn)
        # False discovery rate
        FDR = fp/(tp+fp)
        # Overall accuracy
        ACC_conf = (tp+tn)/(tp+fp+fn+tn)

        tpr_lm = np.append(tpr_lm, TPR)
        tnr_lm = np.append(tnr_lm, TNR)
        ppv_lm = np.append(ppv_lm, PPV)
        npv_lm = np.append(npv_lm, NPV)
        fpr_lm = np.append(fpr_lm, FPR)
        fnr_lm  = np.append(fnr_lm, FNR)
        fdr_lm = np.append(fdr_lm, FDR)
        acc_conf_lm = np.append(acc_conf_lm, ACC_conf)

        # Accuracy:
        acc = accuracy_score(y_test, y_pred_classifier)
        acc_list_metrics= np.append(acc_list_metrics,acc)


        # Classification report:
        # Source: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format
        # Function
        def classification_report_csv(report,rd_state):
            report_data = []
            lines = report.split('\n')
            for line in lines[2:-5]:
                row = {}
                
                row_data = ' '.join(line.split())   
                row_data = row_data.split(' ')
                
                row['class'] = row_data[0]
                row['precision'] = float(row_data[1])
                row['recall'] = float(row_data[2])
                row['f1_score'] = float(row_data[3])
                row['support'] = float(row_data[4])
                report_data.append(row)
                # return report_data #https://stackoverflow.com/questions/3052793/python-output-from-functions

            dataframe = pd.DataFrame.from_dict(report_data)
            dataframe.to_csv('classification_report_9Oct2022_'+ str(rd_state) + 'RF_'+str(name_of_indice)+ '.csv', index = False)
        
        report = classification_report(y_test, y_pred_classifier)
        classification_report_csv(report,i) # Previously I assigned it a variable but it returns a csv anyway so I dont need to. 
        
    # It is cheaper to fill out a list than fill out a dataframe: https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-then-filling-it
    df_acc_metrics = pd.DataFrame(acc_list_metrics)
    df_acc_metrics.rename(columns = {"0":"Accuracy"}, inplace = True)
    #df_acc_metrics.rename({'0':'Accuracy_RF_SPI'},)
    df_acc_metrics.to_csv('Random_state_0_to_50_Acc_9Oct_RF'+str(name_of_indice)+'.csv')
        
    df_tn_metrics = pd.DataFrame(tn_list_metrics)
    df_tn_metrics.rename(columns = {"0":"True Negative"}, inplace = True)
    df_tn_metrics.to_csv('Random_state_0_to_50_trueneg_9Oct_RF'+str(name_of_indice)+'.csv')

    df_fp_metrics = pd.DataFrame(fp_list_metrics)
    df_fp_metrics.rename(columns = {"0":"False Positive"}, inplace = True)
    df_fp_metrics.to_csv('Random_state_0_to_50_falsepos_9Oct_RF'+str(name_of_indice)+'.csv')

    df_fn_metrics = pd.DataFrame(fn_list_metrics)
    df_fn_metrics.rename(columns = {"0":"False Negative"}, inplace = True)
    df_fn_metrics.to_csv('Random_state_0_to_50_falseneg_9Oct_RF'+str(name_of_indice)+'.csv')

    df_tp_metrics = pd.DataFrame(tp_list_metrics)
    df_tp_metrics.rename(columns = {"0":"True Positive"}, inplace = True)
    df_tp_metrics.to_csv('Random_state_0_to_50_truepos_9Oct_RF'+str(name_of_indice)+'.csv')  
    

    tpr_lm_df = pd.DataFrame(tpr_lm)
    tpr_lm_df.columns = ['True Positive Rate / Sensitivity_RF_'+str(name_of_indice)]
    #tpr_lm_df.rename(columns = {"0":"True Positive Rate / Sensitivity"}, inplace = True)
    tpr_lm_df.to_csv('Random_state_0_to_50_tpr_9Oct_RF'+str(name_of_indice)+'.csv')

    tnr_lm_df = pd.DataFrame(tnr_lm)
    tnr_lm_df.columns = ['True Negative Rate / Specificity_RF_'+str(name_of_indice)]
    #tnr_lm_df.rename(columns = {"0":"True Negative Rate / Specificity"}, inplace = True)
    tnr_lm_df.to_csv('Random_state_0_to_50_tnr_9Oct_RF'+str(name_of_indice)+'.csv')
    
    ppv_lm_df = pd.DataFrame(ppv_lm)
    ppv_lm_df.columns = ['Positive predictive value / Precision_RF_'+str(name_of_indice)]
    #ppv_lm_df.rename(columns = {"0":"Positive predictive value / Precision"}, inplace = True)
    ppv_lm_df.to_csv('Random_state_0_to_50_ppv_9Oct_RF'+str(name_of_indice)+'.csv')
    
    npv_lm_df = pd.DataFrame(npv_lm)
    npv_lm_df.columns = ['Negative predictive value_RF_'+str(name_of_indice)]
    #npv_lm_df.rename(columns = {"0":"Negative predictive value"}, inplace = True)
    npv_lm_df.to_csv('Random_state_0_to_50_npv_9Oct_RF'+str(name_of_indice)+'.csv')

    fpr_lm_df = pd.DataFrame(fpr_lm)
    fpr_lm_df.columns = ['False Positive rate / Fall out_RF_'+str(name_of_indice)]
    #fpr_lm_df.rename(columns = {"0":"False Positive rate / Fall out"}, inplace = True)
    fpr_lm_df.to_csv('Random_state_0_to_50_fpr_9Oct_RF'+str(name_of_indice)+'.csv')

    fnr_lm_df = pd.DataFrame(fnr_lm)
    fnr_lm_df.columns = ['False Negative rate_RF_'+str(name_of_indice)]
    #fnr_lm_df.rename(columns = {"0":"False Negative rate"}, inplace = True)
    fnr_lm_df.to_csv('Random_state_0_to_50_fnr_9Oct_RF'+str(name_of_indice)+'.csv')

    fdr_lm_df = pd.DataFrame(fdr_lm)
    fdr_lm_df.columns = ['False discovery rate_RF_'+str(name_of_indice)]
    #fdr_lm_df.rename(columns = {"0":"False discovery rate"}, inplace = True)
    fdr_lm_df.to_csv('Random_state_0_to_50_fdr_9Oct_RF'+str(name_of_indice)+'.csv')

    acc_conf_lm_df = pd.DataFrame(acc_conf_lm)
    acc_conf_lm_df.columns = ['Overall accuracy (from conf m)_RF_'+str(name_of_indice)]
    #acc_conf_lm_df.rename(columns = {"0":"Overall accuracy (from conf m)"}, inplace = True)
    acc_conf_lm_df.to_csv('Random_state_0_to_50_acc_conf_9Oct_RF'+str(name_of_indice)+'.csv')


    ##Combining classification reports
    import glob
    import os
    ##Setting path of files and giving name
    files = os.path.join("classification_report_9Oct2022*.csv")
    ##List of merged files returned
    files = glob.glob(files)
    ## Join files with concat
    df_files = pd.concat(map(pd.read_csv, files), ignore_index=True)
    # print(df_files)
    ##Saving to csv
    df_files.to_csv('Random_state_0_to_50_Classification_Report_Final_9Oct_RF_' + str(name_of_indice)+'.csv')


    # # To just combine accuracy and confusion matrix    
    ## Combining Acc and Confusion Matrix: Source:https://www.statology.org/pandas-merge-multiple-dataframes/
    from functools import reduce

    #define list of DataFrames
    dfs = [df_acc_metrics, df_fp_metrics, df_fn_metrics, df_tp_metrics , df_tn_metrics, tpr_lm_df,tnr_lm_df,ppv_lm_df,npv_lm_df,fpr_lm_df,fnr_lm_df,fdr_lm_df,acc_conf_lm_df]

    #merge all DataFrames into one
    #final_df = reduce(lambda  left,right: pd.merge(left,right,how='left'), dfs)
    # Trying to merge on index singe the above merges it by attaching it below. 
    final_df = reduce(lambda left,right: pd.merge(left,right,left_index = True,right_index=True, how='outer'), dfs)

    final_df.to_csv('Combined_Metrics_Accuracy_Conf_Matrix_X_test_for_RF_for_'+str(name_of_indice)+'_random_state_0_to_50_9Oct.csv')




In [ ]:
# Opening database with indice values
db = pd.read_csv('Database_All_AWRA_NDVI_MOf_and_3MPrecip_noWel_postcode_MERGED_CDI_data_manual_check_2022_10_05.csv') 
db.head()
# This below is specific for SPI
db.drop(columns = ['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0','Unnamed: 0.1'],inplace=True)
get_metrics_for_TEST_data(db,'CDI')

Combining CDI and RF

In [ ]:
RF_metrics = pd.read_csv('Combined_Metrics_Accuracy_Conf_Matrix_X_test_for_RF_for_CDI_random_state_0_to_50_9Oct.csv')
CDI_metrics = pd.read_csv('All_Combined_Metrics_Accuracy_Conf_Matrix_Rates_CDI_all_thresholds_Final_10Oct.csv')

# # To just combine    
## Source:https://www.statology.org/pandas-merge-multiple-dataframes/
from functools import reduce

#define list of DataFrames
dfs = [RF_metrics,CDI_metrics]

# Trying to merge on index singe the above merges it by attaching it below. 
final_df = reduce(lambda left,right: pd.merge(left,right,left_index = True,right_index=True, how='outer'), dfs)

final_df.to_csv('Combined_Metrics_Accuracy_Conf_Matrix_for_CDI_and_RF_random_state_0_to_50_10Oct.csv')



Plotting

In [ ]:
# Redoing the plots without Recall and Precision
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Looping through 
# Making a tuple that has the values in the column name we want to remove
column_names = ['CDI_0','CDI_01','CDI_012','CDI_0123','CDI_01234'] #This is a tuple. startswith only takes a tuple of strings
# __contains only takes one str
for cdi_name_value in column_names:
    # Reading in file and removing unnamed columns and overall accuracy
    cdi_rf_metrics = pd.read_csv('Combined_Metrics_Accuracy_Conf_Matrix_for_CDI_and_RF_random_state_0_to_50_10Oct.csv')
    # Getting a list of column names:
    metrics_col_name_list = cdi_rf_metrics.columns.tolist()
    # Making a tuple that has the start of the column name we want to remove
    unnamed_ov_acc_list = ('Unnamed:','Overall accuracy') #This is a tuple. startswith only takes a tuple of strings
    #metrics_col_name_list
    # creating a list of all the col names that start with Unnamed
    all_unnamed = [x for x in metrics_col_name_list if x.startswith(unnamed_ov_acc_list)]
    #print(all_unnamed)
    cdi_rf_metrics.drop(columns=all_unnamed,inplace=True)
    # Now removing all of these
    metrics_col_name_list = cdi_rf_metrics.columns.tolist()
    tn_fn_tp_fp_names = ('True Positive_CDI','False Positive_CDI','True Negative_CDI','False Negative_CDI','True Positive_RF','False Positive_RF','True Negative_RF','False Negative_RF','True Positive Rate','Positive predictive value')
    # creating a list of all the col names that start with Unnamed
    all_contains = [x for x in metrics_col_name_list if x.startswith(tn_fn_tp_fp_names)]
    cdi_rf_metrics.drop(columns=all_contains,inplace=True)

    # Getting a list of column names in dataframe:
    metrics_col_name_list = cdi_rf_metrics.columns.tolist()

    # Creating tuple of SPI names in dataframe and removing the SPI value I want to plot
    c_names = ['CDI_0','CDI_01','CDI_012','CDI_0123','CDI_01234']  #This is a tuple. startswith only takes a tuple of strings
    c_names.remove(cdi_name_value) #https://www.studytonight.com/post/python-how-to-remove-a-specific-element-from-a-list-of-tuples
    print(c_names)

    for column_n in c_names:
        # creating a list of all the col names that contain the above
        all_contains = [x for x in metrics_col_name_list if x.endswith(column_n)]
        cdi_rf_metrics.drop(columns=all_contains,inplace=True)
        
    # Assigning it to a new db name
    rf_cdi_db = cdi_rf_metrics

    # Rearranging columns alphabetically
    # https://iqcode.com/code/python/how-to-sort-columns-alphabetically-in-pandas
    # Creating a list of column names
    col_name = list(rf_cdi_db.columns)
    print(col_name)
    # Sorting alphabetically
    col_name.sort()
    
    # Sorting columnas and reassigning to the dataframe
    rf_cdi_db = rf_cdi_db[col_name]
    # Renaming columns so the labels in the figures are shorter and only the title says what CDI it is
    # Example of what one is: ['Accuracy_CDI_0', 'Accuracy_RF', 'False Negative rate_CDI_0', 'False Negative rate_RF_CDI', 'False Positive rate / Fall out_CDI_0', 'False Positive rate / Fall out_RF_CDI', 'False discovery rate_CDI_0', 'False discovery rate_RF_CDI', 'Negative predictive value_CDI_0', 'Negative predictive value_RF_CDI', 'Positive predictive value / Precision_CDI_0', 'Positive predictive value / Precision_RF_CDI', 'True Negative Rate / Specificity_CDI_0', 'True Negative Rate / Specificity_RF_CDI', 'True Positive Rate / Sensitivity_CDI_0', 'True Positive Rate / Sensitivity_RF_CDI']
    rf_cdi_db.columns = ['Accuracy (CDI)', 'Accuracy (RF)', 'FN rate (CDI)', 'FN Rate (RF)', 'FP rate (CDI)', 'FP rate (RF)', 'FDR (CDI)', 'FDR (RF)', 'NPV (CDI)', 'NPV (RF)', 'TN Rate (CDI)', 'TN Rate (RF)'] # https://fedingo.com/how-to-rename-columns-in-pandas/
   
    # Creating figure
    sns.set(style='whitegrid')
    #facecolor = '#eaeaf2'
    fig, ax = plt.subplots(figsize=(12, 7))#, facecolor=facecolor)

    # Specifying color of boxplot RF and CDI
    #create your own color array
    my_colors =['#F5793A','#3498db']#["#ffa500", "#3498db"] #["#2ecc71", "#006a4e"]
    # Useful links: https://www.geeksforgeeks.org/how-to-use-seaborn-color-palette-to-color-boxplot/
    # If the color palette has fewer colours then it repeats themselves. This allows every two to be differet: https://www.geeksforgeeks.org/python-plotly-how-to-set-up-a-color-palette/ 
    
    # Create box plot:
    ax = sns.boxplot(data=rf_cdi_db, 
                    palette= my_colors, 
                    linewidth=1.2, 
                    fliersize=2,
                    flierprops=dict(marker='o', markersize=4)) 

    # Getting a list of all column names:
    Col_names = list(rf_cdi_db.columns.values)
    print(Col_names)

    # Rotating the x labels - Source: https://www.geeksforgeeks.org/rotate-axis-tick-labels-in-seaborn-and-matplotlib/
    ax.set_xticklabels(labels=Col_names, rotation=90) 
    
    # Set labels font parameter
    font_color = '#525252'
    csfont = {'fontname':'Georgia'}
    hfont = {'fontname':'Calibri'}

    ax.set_ylabel('Score', fontsize=16, color=font_color, **hfont)
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set(fontsize=16, color=font_color, **hfont)

    # Setting the range of the y axis 
    ax.set(ylim=(-0.01, 1.01)) #https://stackoverflow.com/questions/33227473/how-to-set-the-range-of-y-axis-for-a-seaborn-boxplot

    # Create title:
    title = cdi_name_value
    fig.suptitle(title, y=.97, fontsize=22, color=font_color, **csfont)
    #subtitle = 'Random state: 0 to 50'
    #plt.title(subtitle, fontsize=18, pad=10, color=font_color, **hfont)
    plt.subplots_adjust(top=0.85)

    # Set colour of outlier points: 
    for i, box in enumerate(ax.artists):
        col = box.get_facecolor()
        plt.setp(ax.lines[i*6+5], mfc=col, mec=col)


    # But there are values at 0 which are not 0. I want to have more than 0.0 but 3 dec places. 
    # Potential other way: https://pyquestions.com/labeling-boxplot-in-seaborn-with-median-value 
    # Interesting plotting: https://stackoverflow.com/questions/60995546/box-plot-with-divisor-in-seaborn-python
    # Will do it later.

    # Adding legend:
    #handles, _ = ax.get_legend_handles_labels()
    #ax.legend(handles,['CDI','RF'],loc = 'best') # https://stackoverflow.com/questions/62069967/adding-a-legend-to-a-boxplot-in-matplotlib-seaborn
    custom_lines = [Line2D([0], [0], color='#F5793A', lw=4),
                    Line2D([0], [0], color='#3498db', lw=4)]
                    
    ax.legend(custom_lines, ['CDI', 'RF']) # Source: https://matplotlib.org/stable/gallery/text_labels_and_annotations/custom_legends.html

    # Making sure that the x labels all fit in the saved figure: https://stackoverflow.com/questions/45239261/matplotlib-savefig-text-chopped-off
    # Saving the figure:
    filename = 'sns-boxplot_RFvs'+ str(cdi_name_value)
    plt.savefig(filename+'7_Nov_22.png',bbox_inches = "tight",dpi = 300)


Plotting recall, precision and F1 score plot

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Looping through 
# Making a tuple that has the values in the column name we want to remove
column_names = ['CDI_0','CDI_01','CDI_012','CDI_0123','CDI_01234'] #This is a tuple. startswith only takes a tuple of strings
# __contains only takes one str

# Reading in file and removing unnamed columns and overall accuracy
cdi_rf_metrics = pd.read_csv('Precision_Recall_F1_score_combined_RF_and_CDI_2_Nov_22.csv')

# Dropping all class 0 and keeping class 1: (https://www.golinuxcloud.com/pandas-drop-rows-examples/)
cdi_rf_metrics = cdi_rf_metrics.drop(cdi_rf_metrics[cdi_rf_metrics['class'] == 0].index)
# Getting a list of column names:
metrics_col_name_list = cdi_rf_metrics.columns.tolist()
# Making a tuple that has the start of the column name we want to remove
unnamed_ov_acc_list = ('Unnamed: 0','support','class') #This is a tuple. startswith only takes a tuple of strings
#metrics_col_name_list
# creating a list of all the col names that start with Unnamed
all_unnamed = [x for x in metrics_col_name_list if x.startswith(unnamed_ov_acc_list)]
#print(all_unnamed)
cdi_rf_metrics.drop(columns=all_unnamed,inplace=True)
    
# Assigning it to a new db name
rf_cdi_db = cdi_rf_metrics

# Rearranging columns alphabetically
# https://iqcode.com/code/python/how-to-sort-columns-alphabetically-in-pandas
# Creating a list of column names
col_name = list(rf_cdi_db.columns)
print(col_name)
# Sorting alphabetically
col_name.sort()

# Sorting columnas and reassigning to the dataframe
rf_cdi_db = rf_cdi_db[col_name]
# Renaming columns so the labels in the figures are shorter and only the title says what CDI it is
# Example of what one is: [[['f1_score_CDI0', 'f1_score_CDI_01', 'f1_score_CDI_012', 'f1_score_CDI_0123', 'f1_score_CDI_01234', 'f1_score_RF', 'precision_CDI0', 'precision_CDI_01', 'precision_CDI_012', 'precision_CDI_0123', 'precision_CDI_01234', 'precision_RF', 'recall_CDI0', 'recall_CDI_01', 'recall_CDI_012', 'recall_CDI_0123', 'recall_CDI_01234', 'recall_RF']]
rf_cdi_db.columns =['F1 score (CDI_0)', 'F1 score (CDI_01)', 'F1 score (CDI_012)', 'F1 score (CDI_0123)', 'F1 score (CDI_01234)', 'F1 score (RF)', 'Precision (CDI_0)', 'Precision (CDI_01)', 'Precision (CDI_012)',
                    'Precision (CDI_0123)', 'Precision (CDI_01234)', 'Precision (RF)', 'Recall (CDI_0)', 'Recall (CDI_01)', 'Recall (CDI_012)', 'Recall (CDI_0123)', 'Recall (CDI_01234)', 'Recall (RF)']
#rf_cdi_db.columns =['F1 score', 'F1 score', 'F1 score', 'F1 score', 'F1 score', 'F1 score', 'Precision', 'Precision', 'Precision', 'Precision', 'Precision', 'Precision', 'Recall', 'Recall', 'Recall', 'Recall', 'Recall', 'Recall']

# Creating figure
sns.set(style='whitegrid')
#facecolor = '#eaeaf2'
fig, ax = plt.subplots(figsize=(12, 7))#, facecolor=facecolor)

# Specifying color of boxplot RF and CDI
#create your own color array
#my_colors = ['#984ea3', '#ff7f00', '#4daf4a',
#                  '#f781bf', '#BDB8AD', '#377eb8'] #Palette from https://gist.github.com/thriveth/8560036 - replaced a65628 with BDB8AD
my_colors =['#BDB8AD','#F5793A','#A95AA1','#4daf4a',
                  '#f781bf','#377eb8']
# Useful links: https://www.geeksforgeeks.org/how-to-use-seaborn-color-palette-to-color-boxplot/
# If the color palette has fewer colours then it repeats themselves. This allows every two to be differet: https://www.geeksforgeeks.org/python-plotly-how-to-set-up-a-color-palette/ 

# Create box plot:
ax = sns.boxplot(data=rf_cdi_db, 
                palette= my_colors, 
                linewidth=1.2, 
                fliersize=2,
                flierprops=dict(marker='o', markersize=4)) 

# Getting a list of all column names:
Col_names = list(rf_cdi_db.columns.values)
print(Col_names)

# Rotating the x labels - Source: https://www.geeksforgeeks.org/rotate-axis-tick-labels-in-seaborn-and-matplotlib/
ax.set_xticklabels(labels=Col_names, rotation=90) 

# Set labels font parameter
font_color = '#525252'
csfont = {'fontname':'Georgia'}
hfont = {'fontname':'Calibri'}

ax.set_ylabel('Score', fontsize=16, color=font_color, **hfont)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set(fontsize=16, color=font_color, **hfont)

# Setting the range of the y axis 
ax.set(ylim=(-0.01, 1.01)) #https://stackoverflow.com/questions/33227473/how-to-set-the-range-of-y-axis-for-a-seaborn-boxplot

# Create title:
title = 'CDI and RF'
fig.suptitle(title, y=.97, fontsize=22, color=font_color, **csfont)
#subtitle = 'Random state: 0 to 50'
#plt.title(subtitle, fontsize=18, pad=10, color=font_color, **hfont)
plt.subplots_adjust(top=0.85)

# Set colour of outlier points: 
for i, box in enumerate(ax.artists):
    col = box.get_facecolor()
    plt.setp(ax.lines[i*6+5], mfc=col, mec=col)


# But there are values at 0 which are not 0. I want to have more than 0.0 but 3 dec places. 
# Potential other way: https://pyquestions.com/labeling-boxplot-in-seaborn-with-median-value 
# Interesting plotting: https://stackoverflow.com/questions/60995546/box-plot-with-divisor-in-seaborn-python
# Will do it later.

# Adding legend:
#handles, _ = ax.get_legend_handles_labels()
#ax.legend(handles,['CDI','RF'],loc = 'best') # https://stackoverflow.com/questions/62069967/adding-a-legend-to-a-boxplot-in-matplotlib-seaborn
custom_lines = [Line2D([0], [0], color='#BDB8AD', lw=4),
                 Line2D([0], [0], color='#F5793A', lw=4),
                 Line2D([0], [0], color='#A95AA1', lw=4),
                 Line2D([0], [0], color='#4daf4a', lw=4),
                 Line2D([0], [0], color='#f781bf', lw=4),
                 Line2D([0], [0], color='#377eb8', lw=4)]
                
ax.legend(custom_lines, ['CDI_0','CDI_01','CDI_012','CDI_0123','CDI_01234', 'RF']) # Source: https://matplotlib.org/stable/gallery/text_labels_and_annotations/custom_legends.html

# Making sure that the x labels all fit in the saved figure: https://stackoverflow.com/questions/45239261/matplotlib-savefig-text-chopped-off
# Saving the figure:
filename = 'sns-boxplot_RFvsCDI_f1_score_P_R'
plt.savefig(filename+'7_Nov_22.png',bbox_inches = "tight",dpi=300)
